In [7]:
import cv2
import numpy as np
import json
import os

# Paths
image_path = "data/x.png"
output_dir = "data"
output_json = os.path.join(output_dir, "parking_zones.json")

# Create data directory if not exists
os.makedirs(output_dir, exist_ok=True)

# Load image
image = cv2.imread(image_path)
output = image.copy()

# Convert to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Blur and edge detection
blur = cv2.GaussianBlur(gray, (5,5), 0)
edges = cv2.Canny(blur, 50, 150)

# Find contours (potential parking slots)
contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

parking_zones = []
zone_id = 1

for cnt in contours:
    x, y, w, h = cv2.boundingRect(cnt)

    # Filter small or large areas
    if 40 < w < 150 and 80 < h < 200:
        roi = gray[y:y+h, x:x+w]
        mean_val = np.mean(roi)

        # Assume lighter areas mean free space
        status = "free" if mean_val > 100 else "occupied"

        parking_zones.append({
            "zone_id": f"A{zone_id}",
            "coordinates": {"x1": int(x), "y1": int(y), "x2": int(x+w), "y2": int(y+h)},
            "status": status
        })
        zone_id += 1

# Save JSON to data/ folder
with open(output_json, "w") as f:
    json.dump({"parking_zones": parking_zones}, f, indent=2)

print(f"✅ Parking data saved to {output_json}")

✅ Parking data saved to data/parking_zones.json


In [9]:
# =============================================================================
# STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION (FIXED)
# =============================================================================

# !pip install ultralytics opencv-python numpy

print("\n🎥 STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION")

import cv2
import json
import numpy as np
from ultralytics import YOLO
from datetime import datetime
import os


class ParkingVisualizer:
    def __init__(self, model_path, region_json, video_path, output_path):
        """
        YOLOv8-based Parking Visualization System
        """
        self.model = YOLO(model_path)
        self.video_path = video_path
        self.output_path = output_path

        with open(region_json, "r") as f:
            data = json.load(f)

        # Accept both "parking_zones" key or direct list
        self.regions = data["parking_zones"] if isinstance(data, dict) and "parking_zones" in data else data

        self.colors = {
            "occupied": (0, 0, 255),  # Red
            "available": (0, 255, 0),  # Green
        }

    def is_inside_region(self, point, polygon):
        """Check if a detected object's center point lies inside a polygon region."""
        return cv2.pointPolygonTest(np.array(polygon, np.int32), point, False) >= 0

    def run(self, show_live=False):
        """Run visualization process."""
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print("❌ Error: Unable to open video file.")
            return

        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = cap.get(cv2.CAP_PROP_FPS)

        os.makedirs(os.path.dirname(self.output_path), exist_ok=True)
        out = cv2.VideoWriter(
            self.output_path,
            cv2.VideoWriter_fourcc(*"mp4v"),
            fps,
            (frame_width, frame_height),
        )

        print("🚗 Processing video frames for parking visualization...")
        frame_count = 0
        total_spots = len(self.regions)
        start_time = datetime.now()

        while True:
            ret, frame = cap.read()
            if not ret:
                break

            detections = self.model(frame, verbose=False)
            boxes = detections[0].boxes.xyxy.cpu().numpy() if detections and detections[0].boxes else []

            region_status = {str(i): "available" for i in range(len(self.regions))}

            for box in boxes:
                x1, y1, x2, y2 = map(int, box[:4])
                cx, cy = int((x1 + x2) / 2), int((y1 + y2) / 2)

                for i, region in enumerate(self.regions):
                    # Handle both "points" and "coordinates" formats
                    if "points" in region:
                        polygon = region["points"]
                    elif "coordinates" in region:
                        c = region["coordinates"]
                        polygon = [
                            [c["x1"], c["y1"]],
                            [c["x2"], c["y1"]],
                            [c["x2"], c["y2"]],
                            [c["x1"], c["y2"]],
                        ]
                    else:
                        continue

                    if self.is_inside_region((cx, cy), polygon):
                        region_status[str(i)] = "occupied"

            occupied = list(region_status.values()).count("occupied")
            available = total_spots - occupied
            occupancy_rate = (occupied / total_spots) * 100 if total_spots > 0 else 0

            # Draw all regions
            for i, region in enumerate(self.regions):
                if "points" in region:
                    pts = np.array(region["points"], np.int32)
                elif "coordinates" in region:
                    c = region["coordinates"]
                    pts = np.array(
                        [
                            [c["x1"], c["y1"]],
                            [c["x2"], c["y1"]],
                            [c["x2"], c["y2"]],
                            [c["x1"], c["y2"]],
                        ],
                        np.int32,
                    )
                else:
                    continue

                status = region_status[str(i)]
                color = self.colors[status]
                cv2.polylines(frame, [pts], True, color, 2)
                cv2.putText(
                    frame,
                    status.upper(),
                    (pts[0][0], pts[0][1] - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2,
                )

            # Draw info box
            cv2.rectangle(frame, (10, 10), (380, 80), (50, 50, 50), -1)
            cv2.putText(
                frame,
                f"Total Spots: {total_spots}",
                (20, 30),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 255, 255),
                2,
            )
            cv2.putText(
                frame,
                f"Available: {available}  Occupied: {occupied}",
                (20, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0, 255, 255),
                2,
            )
            cv2.putText(
                frame,
                f"Occupancy: {occupancy_rate:.1f}%",
                (200, 70),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (255, 255, 255),
                2,
            )

            out.write(frame)
            frame_count += 1

            if show_live:
                try:
                    cv2.imshow("Smart Parking Visualization", frame)
                    if cv2.waitKey(1) & 0xFF == ord("q"):
                        break
                except cv2.error:
                    print("⚠️ GUI not supported — skipping live display.")
                    show_live = False

            if frame_count % 50 == 0:
                print(f"🧩 Processed {frame_count} frames...")

        cap.release()
        out.release()

        if show_live:
            try:
                cv2.destroyAllWindows()
            except cv2.error:
                print("⚠️ GUI not supported — skipping window cleanup.")

        duration = (datetime.now() - start_time).seconds
        print(f"✅ Visualization complete — saved to {self.output_path}")
        print(f"🖼️ Processed {frame_count} frames in {duration}s")
        print(f"📊 Final Occupancy Rate: {occupancy_rate:.1f}% ({occupied}/{total_spots} occupied)")
# =============================================================================
# Example Usage
# =============================================================================

model_path = "models/yolov8n.pt"
region_json = "data/parking_zones.json"
video_path = "data/sample_video.mp4"
output_path = "data/parking_visual.mp4"  # ✅ save in data/

visualizer = ParkingVisualizer(model_path, region_json, video_path, output_path)
visualizer.run(show_live=False)  # ✅ Set to True only if your environment supports OpenCV windows


🎥 STEP 4: VISUAL AI DEMO – YOLOv8 PARKING MANAGEMENT VISUALIZATION
🚗 Processing video frames for parking visualization...
🧩 Processed 50 frames...
🧩 Processed 100 frames...
🧩 Processed 150 frames...
🧩 Processed 200 frames...
✅ Visualization complete — saved to data/parking_visual.mp4
🖼️ Processed 206 frames in 51s
📊 Final Occupancy Rate: 0.0% (0/9 occupied)
